# 📘 Агентные Архитектуры 2: Tool Use (Использование инструментов)

Наш второй ноутбук охватывает одну из самых трансформационных агентных архитектур: **Tool Use**. Этот паттерн является мостом, который соединяет способности к рассуждению Large Language Model (LLM) с реальным, динамичным миром.

Без инструментов LLM — это закрытая система, ограниченная знаниями, "замороженными" в ее обучающих данных. Она не может знать погоду на сегодня, текущую цену акций или статус заказа в базе данных вашей компании. Предоставляя агенту возможность использовать инструменты, мы наделяем его способностью преодолеть это фундаментальное ограничение, позволяя ему делать запросы к API, искать в базах данных и получать доступ к живой информации для предоставления ответов, которые являются не просто обоснованными, но также фактическими, своевременными и релевантными.

### Определение
Архитектура **Tool Use** оснащает агента на базе LLM способностью вызывать внешние функции или API ("инструменты"). Агент автономно решает, когда запрос пользователя не может быть отвечен только с помощью его внутренних знаний, и определяет, какой инструмент нужно вызвать для поиска необходимой информации.

### Высокоуровневый рабочий процесс

1.  **Receive Query (Получение запроса):** Агент получает запрос от пользователя.
2.  **Decision (Решение):** Агент анализирует запрос и доступные ему инструменты. Он решает, нужен ли инструмент для точного ответа на вопрос.
3.  **Action (Действие):** Если инструмент нужен, агент формирует вызов к этому инструменту (например, конкретную функцию с правильными аргументами).
4.  **Observation (Наблюдение):** Система выполняет вызов инструмента, и результат ("наблюдение") возвращается агенту.
5.  **Synthesis (Синтез):** Агент интегрирует вывод инструмента в свой процесс рассуждения, чтобы сгенерировать финальный, обоснованный ответ для пользователя.

### Когда использовать / Применение
*   **Research Assistants (Исследовательские ассистенты):** Ответы на вопросы, требующие самой актуальной информации, с использованием API веб-поиска.
*   **Enterprise Assistants (Корпоративные ассистенты):** Запросы к внутренним базам данных компании для ответов на вопросы типа "Сколько новых пользователей зарегистрировалось на прошлой неделе?"
*   **Scientific & Mathematical Tasks (Научные и математические задачи):** Использование калькулятора или вычислительного движка, такого как WolframAlpha, для точных вычислений, с которыми LLM часто испытывают трудности.

### Сильные и слабые стороны
*   **Сильные стороны:**
    *   **Factual Grounding (Фактическая обоснованность):** Радикально уменьшает галлюцинации за счет получения реальных, живых данных.
    *   **Extensibility (Расширяемость):** Возможности агента могут постоянно расширяться путем простого добавления новых инструментов.
*   **Слабые стороны:**
    *   **Integration Overhead (Накладные расходы на интеграцию):** Требует тщательной "настройки" для определения инструментов, обработки API ключей и управления потенциальными сбоями инструментов.
    *   **Tool Trust (Доверие к инструментам):** Качество ответа агента зависит от надежности и точности инструментов, которые он использует. Агент должен доверять тому, что его инструменты предоставляют корректную информацию.

## Фаза 0: Основа и Настройка

Как и прежде, мы начинаем с настройки нашей среды. Это включает установку необходимых библиотек и конфигурацию наших API ключей для Nebius, LangSmith и конкретного инструмента, который мы будем использовать.

### Шаг 0.1: Установка основных библиотек

**Что мы будем делать:**
Мы установим наш стандартный набор библиотек для оркестрации (`langchain-nebius`, `langgraph`), управления средой (`python-dotenv`) и печати (`rich`). Самое главное, мы также установим `tavily-python`, который предоставляет простой в использовании API для мощного инструмента веб-поиска, который мы дадим нашему агенту.

In [1]:
# !pip install -q -U langchain-nebius langchain langgraph rich python-dotenv tavily-python

### Шаг 0.2: Импорт библиотек и настройка ключей

**Что мы будем делать:**
Мы импортируем необходимые модули и используем `python-dotenv` для загрузки наших API ключей. Для этого ноутбука нам понадобятся ключи для Nebius (для LLM), LangSmith (для трассировки) и Tavily (для инструмента веб-поиска).

**Требуемое действие:** Создайте файл `.env` в этой директории с вашими ключами:
```
NEBIUS_API_KEY="your_nebius_api_key_here"
LANGCHAIN_API_KEY="your_langsmith_api_key_here"
TAVILY_API_KEY="your_tavily_api_key_here"
```

In [ ]:
import os
import json
from typing import List, Annotated, TypedDict, Optional

# Компоненты LangChain
from langchain_nebius import ChatNebius
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import BaseMessage, ToolMessage
from pydantic import BaseModel, Field

# Компоненты LangGraph
from langgraph.graph import StateGraph, END
from langgraph.graph.message import AnyMessage, add_messages
from langgraph.prebuilt import ToolNode

# Для красивого вывода
from rich.console import Console
from rich.markdown import Markdown

from notebook_utils import setup_environment, visualize_graph

# --- Настройка API ключей и трассировки ---
console = setup_environment(
    project_name="Agentic Architecture - Tool Use (Nebius)",
    required_keys=["NEBIUS_API_KEY", "LANGCHAIN_API_KEY", "TAVILY_API_KEY"]
)

Environment variables loaded and tracing is set up.


## Фаза 1: Определение инструментария агента

Агент способен настолько, насколько способны инструменты, к которым у него есть доступ. В этой фазе мы определим и протестируем конкретный инструмент, который мы дадим нашему агенту: живой веб-поиск.

### Шаг 1.1: Создание и тестирование инструмента веб-поиска

**Что мы будем делать:**
Мы создадим экземпляр инструмента `TavilySearchResults`. Самая важная часть определения инструмента — это его **описание (description)**. LLM использует это описание на естественном языке, чтобы понять, что делает инструмент и когда его следует использовать. Четкое, точное описание необходимо для того, чтобы агент принимал правильные решения. Затем мы протестируем инструмент напрямую, чтобы увидеть, как выглядит его необработанный вывод.

In [ ]:
# Инициализируем инструмент. Мы можем установить максимальное количество результатов, чтобы сохранить контекст кратким.
search_tool = TavilySearchResults(max_results=2)

# Критически важно дать инструменту понятное имя и описание для агента
search_tool.name = "web_search"
search_tool.description = "A tool that can be used to search the internet for up-to-date information on any topic, including news, events, and current affairs."

tools = [search_tool]
print(f"Tool '{search_tool.name}' created with description: '{search_tool.description}'")

console = Console()

# Давайте протестируем инструмент напрямую, чтобы увидеть формат его вывода
print("\n--- Testing the tool directly (Тестирование инструмента напрямую) ---")
test_query = "What was the score of the last Super Bowl?"
test_result = search_tool.invoke({"query": test_query})
console.print(f"[bold green]Query (Запрос):[/bold green] {test_query}")
console.print("\n[bold green]Result (Результат):[/bold green]")
console.print(test_result)

Tool 'web_search' created with description: 'A tool that can be used to search the internet for up-to-date information on any topic, including news, events, and current affairs.'

--- Testing the tool directly ---


C:\Users\faree\AppData\Local\Temp\ipykernel_2556\1362582966.py:2: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  search_tool = TavilySearchResults(max_results=2)


Query: What was the score of the last Super Bowl?

Result:

[
    {
        'title': 'List of Super Bowl Winners (1967-2025) - NFL Champions by Year Last 10 Super Bowls Scores | 
StatMuse Super Bowl Winners by Year: Complete List & 2025 Results Super Bowl Winners by Year - ESPN List of Super 
Bowl Winners (1967-2025) - NFL Champions by Year List of Super Bowl champions - Wikipedia Super Bowl Winners by 
Year - ESPN Super Bowl Winners by Year: Complete List & 2025 Results List of Super Bowl champions - Wikipedia List 
of Super Bowl Winners (1967-2025) - NFL Champions by Year Who Played in The Super Bowl Last Year? - Bleacher 
Nation',
        'url': 'https://www.topendsports.com/events/super-bowl/winners-list.htm',
        'content': 'Score: Rams 23, Bengals 20\n Venue: SoFi Stadium, Los Angeles\n Date: February 13, 2022\n MVP: 
Cooper Kupp\n\n### Super Bowls LV & LIV\n\n2021 (LV): Tampa Bay Buccaneers defeated Kansas City Chiefs 31-9, with 
Tom Brady winning his 7th championship.\n\n2020 (LIV): Kansas City Chiefs defeated San Francisco 49ers 31-20, 
ending a 50-year championship drought. [...] | XXVII | 1993 | Dallas | Buffalo | 52-17 | Pasadena |\n| XXVI | 1992 
| Washington | Buffalo | 37-24 | Minneapolis |\n| XXV | 1991 | NY Giants | Buffalo | 20-19 | Tampa |\n| XXIV | 1990
| San Francisco | Denver | 55-10 | New Orleans |\n| XXIII | 1989 | San Francisco | Cincinnati | 20-16 | Miami |\n| 
XXII | 1988 | Washington | Denver | 42-10 | San Diego |\n| XXI | 1987 | NY Giants | Denver | 39-20 | Pasadena |\n| 
XX | 1986 | Chicago | New England | 46-10 | New Orleans | [...] | No. | Year | Winner | Opposition | Score | Venue 
|\n| LIX | 2025 | Philadelphia Eagles | Kansas City Chiefs | 40-22 | New Orleans, Louisiana |\n| LVIII | 2024 | 
Kansas City Chiefs | San Francisco 49ers | 25-22 | Las Vegas, Nevada |\n| LVII | 2023 | Kansas City Chiefs | 
Philadelphia Eagles | 38-35 | Arizona |\n| LVI | 2022 | Los Angeles Rams | Cincinnati Bengals | 23-20 | Los Angeles
|\n| LV | 2021 | Tampa Bay Buccaneers | Kansas City Chiefs | 31-9 | Tampa |',
        'score': 0.7300017
    },
    {
        'title': 'Super Bowl Winners by Year: Complete List & 2025 Results',
        'url': 'https://nflplayoffpass.com/super-bowl-winners/',
        'content': '| Super Bowl | Year | Winner | Opposition | Score | Stadium |\n ---  ---  --- |\n| LIX | 2025 |
Philadelphia Eagles | Kansas City Chiefs | 40-22 | Caesars Superdome |\n| LVIII | 2024 | Kansas City Chiefs | San 
Francisco 49ers | 25–22 (OT) | Allegiant Stadium |\n| LVII | 2023 | Kansas City Chiefs | Philadelphia Eagles | 
38–35 | State Farm Stadium |\n| LVI | 2022 | Los Angeles Rams | Cincinnati Bengals | 23–20 | SoFi Stadium | [...] |
XLV | 2011 | Green Bay Packers | Pittsburgh Steelers | 31–25 | Cowboys Stadium |\n| XLIV | 2010 | New Orleans 
Saints | Indianapolis Colts | 31–17 | Sun Life Stadium |\n| XLIII | 2009 | Pittsburgh Steelers | Arizona Cardinals 
| 27–23 | Raymond James Stadium |\n| XLII | 2008 | New York Giants | New England Patriots | 17–14 | University of 
Phoenix Stadium |\n| XLI | 2007 | Indianapolis Colts | Chicago Bears | 29–17 | Dolphin Stadium | [...] | XXIII | 
1989 | San Francisco 49ers | Cincinnati Bengals | 20–16 | Joe Robbie Stadium |\n| XXII | 1988 | Washington Redskins
| Denver Broncos | 42–10 | Jack Murphy Stadium |\n| XXI | 1987 | New York Giants | Denver Broncos | 39–20 | Rose 
Bowl |\n| XX | 1986 | Chicago Bears | New England Patriots | 46–10 | Louisiana Superdome |\n| XIX | 1985 | San 
Francisco 49ers | Miami Dolphins | 38–16 | Stanford Stadium |\n| XVIII | 1984 | Los Angeles Raiders | Washington 
Redskins | 38–9 | Tampa Stadium |',
        'score': 0.7293082
    }
]

**Обсуждение вывода:**
Тест показывает необработанный вывод нашего инструмента `web_search`. Он возвращает список словарей, где каждый словарь содержит URL и фрагмент контента результата поиска. Эта структурированная информация — именно то, что агент получит в качестве своего "наблюдения" после того, как решит использовать инструмент. Теперь, когда у нас есть функциональный инструмент, мы можем построить агента, который научится его использовать.

## Фаза 2: Построение агента, использующего инструменты, с помощью LangGraph

Теперь мы сконструируем рабочий процесс агента. Это включает в себя ознакомление LLM с инструментами и создание графа, который позволяет ей выполнять цикл "думай-действуй-наблюдай" (think-act-observe), что является сутью использования инструментов.

### Шаг 2.1: Определение состояния графа (Graph State)

**Что мы будем делать:**
Состояние для агента, использующего инструменты, обычно представляет собой список сообщений, отражающий историю разговора. Эта история включает вопросы пользователя, мысли и вызовы инструментов агента, а также результаты работы этих инструментов. Мы будем использовать `TypedDict`, который может хранить любой тип сообщений LangChain.

In [ ]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

print("TypedDict AgentState определен для управления историей разговора.")

AgentState TypedDict defined to manage conversation history.


### Шаг 2.2: Привязка инструментов к LLM

**Что мы будем делать:**
Это критический шаг, на котором мы делаем LLM "осведомленной" об инструментах. Мы используем метод `.bind_tools()`, который передает имена и описания наших инструментов в системный промпт LLM. Это позволяет внутренней логике модели решать, когда вызывать инструмент, основываясь на его описании.

In [ ]:
llm = ChatNebius(model="meta-llama/Meta-Llama-3.1-8B-Instruct", temperature=0)

# Привязываем инструменты к LLM, делая ее осведомленной об инструментах
llm_with_tools = llm.bind_tools(tools)

print("LLM была связана с предоставленными инструментами.")

LLM has been bound with the provided tools.


### Шаг 2.3: Определение узлов агента

**Что мы будем делать:**
Наш граф будет иметь два основных узла:
1.  **`agent_node`:** Это "мозг". Он вызывает LLM с текущей историей разговора. Ответ LLM будет либо окончательным ответом, либо запросом на вызов инструмента.
2.  **`tool_node`:** Это "руки". Он принимает запрос на вызов инструмента от `agent_node`, выполняет соответствующий инструмент и возвращает вывод. Мы будем использовать готовый `ToolNode` из LangGraph для этого.

In [ ]:
def agent_node(state: AgentState):
    """Основной узел, который вызывает LLM для принятия решения о следующем действии."""
    console.print("--- AGENT: Thinking... (АГЕНТ: Думает...) ---")
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": [response]}

# ToolNode — это готовый узел из LangGraph, который выполняет инструменты
tool_node = ToolNode(tools)

print("Узел агента (Agent node) и узел инструментов (Tool node) были определены.")

Agent node and Tool node have been defined.


### Шаг 2.4: Определение условного маршрутизатора

**Что мы будем делать:**
После выполнения `agent_node` нам нужно решить, куда идти дальше. Функция маршрутизатора проверяет последнее сообщение от агента. Если это сообщение содержит атрибут `tool_calls`, это означает, что агент хочет использовать инструмент, поэтому мы направляем его в `tool_node`. Если нет, это означает, что агент предоставил окончательный ответ, и мы можем завершить рабочий процесс.

In [ ]:
def router_function(state: AgentState) -> str:
    """Проверяет последнее сообщение агента, чтобы решить следующий шаг."""
    last_message = state["messages"][-1]
    if last_message.tool_calls:
        # Агент запросил вызов инструмента
        console.print("--- ROUTER: Decision is to call a tool. (МАРШРУТИЗАТОР: Решение вызвать инструмент.) ---")
        return "call_tool"
    else:
        # Агент предоставил окончательный ответ
        console.print("--- ROUTER: Decision is to finish. (МАРШРУТИЗАТОР: Решение завершить.) ---")
        return "__end__"

print("Функция маршрутизатора определена.")

Router function defined.


## Фаза 3: Сборка и запуск рабочего процесса

Теперь мы соединим все компоненты вместе в полный, исполняемый граф и запустим его на запросе, который заставит агента использовать его новую возможность веб-поиска.

### Шаг 3.1: Построение и визуализация графа

**Что мы будем делать:**
Мы создадим `StateGraph` и добавим наши узлы и ребра. Ключевой частью является условное ребро, которое использует нашу `router_function` для создания основного цикла рассуждений агента: `agent -> router -> tool -> agent`.

In [ ]:
graph_builder = StateGraph(AgentState)

# Добавляем узлы
graph_builder.add_node("agent", agent_node)
graph_builder.add_node("call_tool", tool_node)

# Устанавливаем точку входа
graph_builder.set_entry_point("agent")

# Добавляем условные ребра
graph_builder.add_conditional_edges(
    "agent",
    router_function,
)

# Добавляем ребро от узла инструмента обратно к агенту, чтобы завершить цикл
graph_builder.add_edge("call_tool", "agent")

# Компилируем граф
tool_agent_app = graph_builder.compile()

print("Граф агента, использующего инструменты, успешно скомпилирован!")

# Визуализируем граф
visualize_graph(tool_agent_app, console)

Tool-using agent graph compiled successfully!
Graph visualization failed: Install pygraphviz to draw graphs: `pip install pygraphviz`.. Please ensure pygraphviz is installed.


**Обсуждение вывода:**
Скомпилированный граф готов. Визуализация четко показывает цикл рассуждений агента. Процесс начинается в узле `agent`. Условное ребро (представленное ромбом) затем маршрутизирует поток. Если инструмент нужен, он идет в `call_tool`, и вывод подается обратно в `agent` для синтеза. Если инструмент не нужен, процесс идет в `__end__`. Эта структура идеально реализует паттерн Tool Use.

### Шаг 3.2: Сквозное выполнение

**Что мы будем делать:**
Давайте запустим агента с вопросом, на который он не может знать ответ из своих обучающих данных, заставляя его использовать инструмент веб-поиска. Мы будем стримить промежуточные шаги, чтобы наблюдать за развертыванием процесса его рассуждений.

In [ ]:
user_query = "What were the main announcements from Apple's latest WWDC event?"
initial_input = {"messages": [("user", user_query)]}

console.print(f"[bold cyan]🚀 Запуск рабочего процесса Tool Use для запроса:[/bold cyan] '{user_query}'\n")

for chunk in tool_agent_app.stream(initial_input, stream_mode="values"):
    chunk["messages"][-1].pretty_print()
    console.print("\n---\n")

console.print("\n[bold green]✅ Рабочий процесс Tool Use завершен![/bold green]")

🚀 Kicking off Tool Use workflow for request: 'What were the main announcements from Apple's latest WWDC event?'

================================ Human Message =================================

What were the main announcements from Apple's latest WWDC event?


---

--- AGENT: Thinking... ---

--- ROUTER: Decision is to call a tool. ---

...

--- AGENT: Thinking... ---

--- ROUTER: Decision is to finish. ---

================================== Ai Message ==================================

The main announcements from Apple's latest WWDC event include a new design that will inform the next decade of iOS, iPadOS, and macOS development, new features for the iPhone, an overhauled Spotlight interface for the Mac, and updates that make the iPad more like a Mac than ever before. Additionally, Apple introduced a ton of new features and updates across every platform, including iOS 26, iPadOS 26, CarPlay, macOS Tahoe, watchOS 26, visionOS 26, and tvOS 26.


---

✅ Tool Use workflow complete!

## Фаза 4: Оценка

Теперь, когда агент отработал, мы можем оценить его производительность. Для агента, использующего инструменты, нас волнуют две вещи: правильно ли он использовал свои инструменты, и был ли окончательный ответ, синтезированный из вывода инструмента, качественным?

### Шаг 4.1: Анализ трассировки выполнения

**Что мы будем делать:**
Глядя на стриминг вывода из предыдущего шага, мы можем проследить точный процесс мышления агента. Вывод показывает различные типы сообщений (`AIMessage` с `tool_calls`, `ToolMessage` с результатами), которые проходят через состояние графа.

**Обсуждение вывода:**
Трассировка выполнения четко показывает паттерн Tool Use в действии:
1.  Первое напечатанное сообщение — от узла `agent`. Это `AIMessage`, содержащее атрибут `tool_calls`, указывающий, что LLM правильно решила использовать инструмент `web_search`.
2.  Следующее сообщение — это `ToolMessage`. Это вывод от `tool_node` после того, как он выполнил поиск и вернул необработанные результаты.
3.  Финальное сообщение — это еще одно `AIMessage`, но на этот раз без `tool_calls`. Это агент, синтезирующий информацию из `ToolMessage` в связный, финальный ответ для пользователя.
Эта трассировка подтверждает, что логика агента и маршрутизация графа сработали идеально.

### Шаг 4.2: Оценка с помощью LLM-as-a-Judge

**Что мы будем делать:**
Мы создадим LLM-'Судью' (Judge), чтобы предоставить структурированную, количественную оценку производительности агента. Критерии оценки будут адаптированы специально для оценки качества использования инструментов.

In [ ]:
class ToolUseEvaluation(BaseModel):
    """Схема для оценки использования инструментов агентом и финального ответа."""
    tool_selection_score: int = Field(description="Оценка 1-5: выбрал ли агент правильный инструмент для задачи.")
    tool_input_score: int = Field(description="Оценка 1-5: насколько корректным и релевантным был входной сигнал для инструмента.")
    synthesis_quality_score: int = Field(description="Оценка 1-5: насколько хорошо агент интегрировал вывод инструмента в свой финальный ответ.")
    justification: str = Field(description="Краткое обоснование оценок.")

judge_llm = llm.with_structured_output(ToolUseEvaluation)

# Для оценки нам нужно реконструировать полную трассировку разговора
final_answer = tool_agent_app.invoke(initial_input)
conversation_trace = "\n".join([f"{m.type}: {m.content or ''} {getattr(m, 'tool_calls', '')}" for m in final_answer['messages']])

def evaluate_tool_use(trace: str):
    prompt = f"""You are an expert judge of AI agents. Evaluate the following conversation trace based on the agent's tool use on a scale of 1-5. Provide a brief justification.
    
    Conversation Trace:
    ```
    {trace}
    ```
    """
    return judge_llm.invoke(prompt)

console.print("--- Evaluating Tool Use Performance (Оценка производительности использования инструментов) ---")
evaluation = evaluate_tool_use(conversation_trace)
console.print(evaluation.model_dump())

--- AGENT: Thinking... ---

--- ROUTER: Decision is to call a tool. ---

--- AGENT: Thinking... ---

--- ROUTER: Decision is to finish. ---

--- Evaluating Tool Use Performance ---

{
    'tool_selection_score': 5,
    'tool_input_score': 5,
    'synthesis_quality_score': 4,
    'justification': "The AI agent used the web search tool to find relevant information about Apple's latest WWDC 
event. The tool output was well-formed and relevant, providing a good summary of the announcements. However, the AI
agent could have done a better job of synthesizing the information, as some of the points mentioned in the output 
are not clearly connected to the main announcements. For example, the mention of the Apple Watch and other devices 
seems out of place in the context of the main announcements. Overall, the AI agent's tool use was effective, but 
could be improved with better synthesis of the information."
}

**Обсуждение вывода:**
Оценка "LLM-как-судья" предоставляет структурированную и обоснованную оценку производительности нашего агента. Высокие баллы по всем трем категориям — `tool_selection_score`, `tool_input_score` и `synthesis_quality_score` — подтверждают, что наш агент не просто использует инструменты, но использует их *эффективно*. Он правильно определил необходимость веб-поиска, сформулировал релевантный запрос и успешно синтезировал полученные факты в полезный и точный окончательный ответ. Эта автоматизированная оценка дает нам уверенность в надежности нашей реализации.

## Заключение

В этом ноутбуке мы построили полного, функционирующего агента на основе архитектуры **Tool Use**. Мы успешно оснастили LLM на базе Nebius инструментом веб-поиска и использовали LangGraph для создания надежного цикла рассуждений, который позволяет агенту решать, когда и как его использовать.

Сквозное выполнение и последующая оценка демонстрируют огромную ценность этого паттерна. Подключив нашего агента к живой, внешней информации, мы фундаментально преодолели ограничение статических обучающих данных. Агент больше не просто мыслитель; он исследователь, способный предоставлять ответы, которые являются обоснованными, фактическими и актуальными. Эта архитектура является фундаментальным строительным блоком для создания практически любого практического ИИ-ассистента реального мира.